## Fluctuation test

A question that has emerged a number of times in evolutionary analysis is whether mutation rates vary, and specifically whether they vary in response to a selection pressure. In particular, do mutations emerge as a result of a stressor or do they occur during the growth of the culture? Even more controversially: does the likelihood of an advantageous mutation increase when bacteria are subjected to some stressor (e.g. antibiotic, bacteriophage, etc.)? For a bit of a history of this problem you can consult https://en.wikipedia.org/wiki/Luria%E2%80%93Delbr%C3%BCck_experiment

In the 1940's Salvatore Luria (experimental biologist) and Max Delbrück (physicist) designed an experiment to distinguish whether mutations arose pre or post-stressor. The experiment was the following: bacterial cultures were grown independently from small inocula. After some period of growth, equal volumes of these cultures were plated at low density onto agar containing the T1 phage, so that only resistant bacteria will end up forming small, visible colonies that could be counted. 

What is the expected distribution of the number of resistant bacteria on a plate if the virus **induced** resistance?

What is the expected distribution if the virus **selected** the resistant bacteria from the population that was plated?

Implement a simulation of this experiment: 
- grow populations of bacteria from a single cell to a specified final number
- consider implementing an event-driven simulation, where cell divisions occur and the time to division can be chosen from a variety of distributions when a cell is born
- implement mutations (mutation rate a parameter) at cell division
- implement the phenotype check (resistant/sensitive)
- derive the distribution of the number of mutants per culture
- explore the form of the distribution when mutations are induced or selected
- explore the form of the distribution for different lifespan distributions (what distribution of lifespans did Luria and Delbrück assume in their model?)